In [2]:
from zipfile import ZipFile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon

In [5]:
az_data = np.load('arizona_matrix.npz', allow_pickle=True)
group_std = az_data['matrix']
group_pops = az_data['pops']

In [7]:
group_std = group_std.astype('float64')

ValueError: could not convert string to float: 'PLST'

In [31]:

def population_agglomerate(matrix, n, populations):
    pop_limit = sum(populations) / n
    clusters = {i: {'members': [i], 'pop':populations[i]} for i in range(len(matrix))}
    distances = {(i,j): np.square(clusters[i]['members'][0] - clusters[j]['members'][0]).sum() for i in range(len(matrix)) for j in range(i+1,len(matrix))}
    idx = len(clusters)
    while len(clusters) > n:
        print('{0: < 10}'.format(len(clusters)), end='\r')
        # Find the closest pair of clusters that don't exceed the pop_limit
        i,j = min(distances.items(), key=lambda x: x[1])[0]
        if clusters[i]['pop'] + clusters[j]['pop'] > pop_limit:
            break

        for c2 in clusters:
            if c2 == i or c2 == j:
                continue
            total_pop = clusters[i]['pop'] + clusters[j]['pop'] + clusters[c2]['pop']
            if total_pop > pop_limit:
                distances[(c2,idx)] = np.inf
            else:
                distances[(c2,idx)] = (clusters[i]['pop'] + clusters[c2]['pop'])*distances[tuple(sorted([i,c2]))] / total_pop  +  (clusters[j]['pop'] + clusters[c2]['pop'])*distances[tuple(sorted([j,c2]))] / total_pop -  clusters[c2]['pop'] *distances[tuple(sorted([i,j]))] / total_pop
        clusters[idx] = {'members': clusters[i]['members'] + clusters[j]['members'], 'pop': clusters[i]['pop'] + clusters[j]['pop']}   
        to_remove = []
        for k in distances:
            if i in k or j in k:
                to_remove.append(k)
        for k in to_remove:
            distances.pop(k)
        clusters.pop(i)
        clusters.pop(j)
        idx += 1
    return clusters

In [32]:
group_pops = np.array(group_data['POP100'].array)
clusts = population_agglomerate(group_std, 141, group_pops)

MemoryError: 

In [33]:
np.savez('maryland_matrix', matrix=group_data, pops=group_pops)

In [ ]:
def consolidate_clusters(matrix, clusters, n, populations):
    pop_limit = sum(populations) / n
    